In [1]:
print ("============================================================")
print (" Course Work                              Robiul Islam      ")
print ("============================================================")


import pandas as pd 
from sklearn.utils import shuffle
from itertools import combinations
import bisect
import math

data = pd.read_csv("datafull.csv",sep=';') 
gen = pd.read_csv('/Users/HSE/coursework/python_vmv/SeqDem/gen.csv')
data['Genaration']=gen
# Remove Null value
data.dropna(inplace = True)
data = shuffle(data, random_state=7)
data.head()
def power_set(List):
    PS = [list(j) for i in range(len(List)) for j in combinations(List, i+1)]
    return PS

def shapley_calc(N,characteristic_function,n):
    shapley_values = []
    for i in range(n):
        shapley = 0
        for j in N:
            if i not in j:
                try:
                    cmod = len(j)
                    Cui = j[:]
                    l = N.index(j)
                    k = N.index(Cui)
                    temp = float(float(characteristic_function[k]) - float(characteristic_function[l])) *\
                               float(math.factorial(cmod) * math.factorial(n - cmod - 1)) / float(math.factorial(n))
                    shapley += temp
                except:
                    temp = float(N[0][0])
                    shapley += temp

        shapley_values.append(shapley)
    return shapley_values 

result =[]
for row in range(data.shape[0]):
    PS = power_set(data.iloc[row])
    Chan_fun =[]
    for i in PS:
        Chan_fun.append(max(i))
    Value = shapley_calc(PS,Chan_fun,5)
    result.append(max(Value))
    
data['result'] = result
print(data.head())

# gen = pd.read_csv('/Users/HSE/coursework/python_vmv/SeqDem/gen.csv')
# print(gen.head())

 Course Work                              Robiul Islam      
      work  separation  partner  marriage  children  Genaration  result
283   18.0        20.0     20.0      21.0      22.0           6   126.0
1329  17.0        25.0     20.0      20.0      29.0           4   119.0
2051  25.0        25.0     25.0      34.0      35.0           5   175.0
450   17.0        19.0     30.0      24.0      26.0           4   119.0
1135  19.0        19.0     23.0      24.0      25.0           2   133.0


In [3]:
print ("============================================================")
print (" Course Work     Start Deep Learning      Robiul Islam      ")
print ("============================================================")

X = data[['work','separation','partner','marriage','children','result']].values
y = data['Genaration'].values


from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder_X_1 = LabelEncoder()
X[:, 1] = labelencoder_X_1.fit_transform(X[:, 1])
labelencoder_X_2 = LabelEncoder()
X[:, 2] = labelencoder_X_2.fit_transform(X[:, 2])

onehotencoder = OneHotEncoder(categorical_features = [1])
X = onehotencoder.fit_transform(X).toarray()
X = X[:, 1:]

# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

# Importing the Keras libraries and packages
import keras
from keras.models import Sequential
from keras.layers import Dense

#Initializing Neural Network
classifier = Sequential()

# Adding the input layer and the first hidden layer
classifier.add(Dense(output_dim = 6, init = 'uniform', activation = 'relu', input_dim = 11))
# Adding the second hidden layer
classifier.add(Dense(output_dim = 6, init = 'uniform', activation = 'relu'))
# Adding the output layer
classifier.add(Dense(output_dim = 1, init = 'uniform', activation = 'sigmoid'))

# Compiling Neural Network
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

# Fitting our model 
classifier.fit(X_train, y_train, batch_size = 7, nb_epoch = 100)

# Predicting the Test set results
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

# Creating the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)

 Course Work     Start Deep Learning      Robiul Islam      


/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:390: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:38: UserWarning: Update your `Dense` call to the Keras 2 API: `

ValueError: Error when checking input: expected dense_4_input to have shape (11,) but got array with shape (26,)

In [185]:
print(x_train.shape)

(247, 4)


In [196]:
import numpy
numpy.unique(len(y))


array([309])

In [211]:

print(data.head())
print(data['work','separation'])

      work  separation  partner  marriage  children  Genaration  result
283   18.0        20.0     20.0      21.0      22.0           6   126.0
1329  17.0        25.0     20.0      20.0      29.0           4   119.0
2051  25.0        25.0     25.0      34.0      35.0           5   175.0
450   17.0        19.0     30.0      24.0      26.0           4   119.0
1135  19.0        19.0     23.0      24.0      25.0           2   133.0


KeyError: ('work', 'separation')

In [214]:
print(y)
import numpy
numpy.unique(y)
print(data[['work','separation','partner','marriage','children','result']].values)

[6 4 5 4 2 0 5 5 6 5 0 4 4 4 4 1 6 6 4 3 6 2 6 5 6 6 5 6 4 6 3 6 5 2 6 5 6
 0 4 3 4 4 4 5 4 5 0 0 3 5 4 3 5 3 4 1 6 6 2 3 3 5 6 1 3 4 4 6 4 5 4 4 4 5
 4 2 1 5 6 6 6 4 5 3 5 6 6 4 4 5 5 5 5 4 6 6 4 3 4 4 6 6 5 3 5 6 3 3 2 0 2
 5 3 3 4 4 0 4 3 4 2 1 0 5 5 3 6 0 5 6 3 3 6 5 4 6 5 2 5 5 6 5 4 6 2 2 5 4
 5 5 0 3 6 3 5 0 4 5 5 0 3 4 5 4 5 5 6 4 5 6 2 4 6 5 5 1 0 5 4 3 5 5 5 3 3
 6 5 0 6 4 1 5 5 6 6 6 6 6 4 6 2 6 4 4 4 3 5 0 4 2 3 6 3 6 6 6 6 4 6 3 4 4
 5 2 2 6 5 6 2 2 5 3 5 6 2 6 5 5 6 4 4 2 5 6 3 3 6 5 6 4 5 6 5 6 4 5 2 6 5
 0 3 6 6 5 5 2 6 0 4 3 6 3 4 2 4 4 5 5 5 0 4 4 5 2 4 5 6 6 5 2 0 3 2 5 4 6
 3 2 5 4 5 5 3 2 2 5 5 0 5]
[[ 18.  20.  20.  21.  22. 126.]
 [ 17.  25.  20.  20.  29. 119.]
 [ 25.  25.  25.  34.  35. 175.]
 ...
 [ 20.  16.  36.  16.  17. 140.]
 [ 15.  18.  15.  21.  21. 105.]
 [ 17.  15.  17.  17.  18. 119.]]
